In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras

model = keras.models.load_model('Model_v2.keras')
GNIFTY_200D_DATA_PATH = '../Data/Past_200D_Data.csv'

In [2]:
from datetime import datetime, timedelta

now = datetime.now()
time_delta = timedelta(days=200)
start = now - time_delta
end = now

print(f"From: {start} to {end}")

From: 2025-03-28 14:45:53.667504 to 2025-10-14 14:45:53.667504


In [3]:
ticker = "^NSEI"
df_Nifty = yf.download(ticker, start, end)
# Calculate RSI (Relative Strength Index)
def calculate_rsi(data, window=14):
    diff = data.diff(1).dropna()
    gain = diff.mask(diff < 0, 0)
    loss = diff.mask(diff > 0, 0).abs()
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df_Nifty['RSI'] = calculate_rsi(df_Nifty['Close'])

# Calculate MACD (Moving Average Convergence Divergence)
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    short_ema = data.ewm(span=short_window, adjust=False).mean()
    long_ema = data.ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal_line = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal_line

macd, signal_line = calculate_macd(df_Nifty['Close'])
df_Nifty['MACD'] = macd
df_Nifty['MACD_Signal'] = signal_line

# Calculate EMA 50
df_Nifty['EMA_50'] = df_Nifty['Close'].ewm(span=50, adjust=False).mean()

# Calculate SMA 20
df_Nifty['SMA_20'] = df_Nifty['Close'].rolling(window=20).mean()

# Calculate Stochastic RSI
def calculate_stoch_rsi(data, window=14, k_window=3, d_window=3):
    rsi = calculate_rsi(data, window)
    min_rsi = rsi.rolling(window=window).min()
    max_rsi = rsi.rolling(window=window).max()
    stoch_rsi = (rsi - min_rsi) / (max_rsi - min_rsi)
    k_line = stoch_rsi.rolling(window=k_window).mean()
    d_line = k_line.rolling(window=d_window).mean()
    return k_line, d_line

df_Nifty['StochRSI_K'], df_Nifty['StochRSI_D'] = calculate_stoch_rsi(df_Nifty['Close'])

df_Nifty.columns = [col for col, _ in df_Nifty.columns]
print(df_Nifty.columns)

C:\Users\Krish Patel\AppData\Local\Temp\ipykernel_18384\454345099.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_Nifty = yf.download(ticker, start, end)
[*********************100%***********************]  1 of 1 completed

Index(['Close', 'High', 'Low', 'Open', 'Volume', 'RSI', 'MACD', 'MACD_Signal',
       'EMA_50', 'SMA_20', 'StochRSI_K', 'StochRSI_D'],
      dtype='object')


In [4]:
df_Nifty

,Close,High,Low,Open,Volume,RSI,MACD,MACD_Signal,EMA_50,SMA_20,StochRSI_K,StochRSI_D
Date,,,,,,,,,,,,
2025-03-28,23519.349609,23649.199219,23450.199219,23600.400391,295400,NaN,0.000000,0.000000,23519.349609,NaN,NaN,NaN
2025-04-01,23165.699219,23565.150391,23136.400391,23341.099609,314300,NaN,-28.211427,-5.642285,23505.480967,NaN,NaN,NaN
2025-04-02,23332.349609,23350.000000,23158.449219,23192.599609,341000,NaN,-36.698851,-11.853599,23498.691502,NaN,NaN,NaN
2025-04-03,23250.099609,23306.500000,23145.800781,23150.300781,283200,NaN,-49.491575,-19.381194,23488.942800,NaN,NaN,NaN
2025-04-04,22904.449219,23214.699219,22857.449219,23190.400391,466800,NaN,-86.523609,-32.809677,23466.021483,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-10-08,25046.150391,25192.500000,25008.500000,25079.750000,227400,40.305099,18.253080,18.234948,24915.657608,25033.237500,0.157538,0.164701
2025-10-09,25181.800781,25199.250000,25024.300781,25074.300781,286300,41.980149,34.127757,21.413510,24926.094595,25043.672559,0.114772,0.147958
2025-10-10,25285.349609,25330.750000,25156.849609,25167.650391,232900,48.623271,54.436567,28.018121,24940.183027,25057.665039,0.160086,0.144132


In [5]:
df_Gnifty = pd.read_csv(GNIFTY_200D_DATA_PATH)
df_Gnifty.head()

,Date,Open,Close,Change(%),High,Low
0,13 Oct 2025,"25,310.50","25,305.00",-0.02%,"25,355.00","25,226.00"
1,12 Oct 2025,"25,205.00","25,205.00",-,"25,205.00","25,205.00"
2,11 Oct 2025,"25,205.00","25,205.00",-,"25,205.00","25,205.00"
3,10 Oct 2025,"25,227.00","25,205.00",-0.09%,"25,468.00","25,205.00"
4,09 Oct 2025,"25,131.50","25,222.50",0.36%,"25,309.00","25,120.50"


In [6]:
df_Gnifty.rename(columns={
    'Close': 'GIFT_Close',
    'Open': 'GIFT_Open',
    'High': 'GIFT_High',
    'Low': 'GIFT_Low',
    'Vol.': 'GIFT_Volume',
    'Change %': 'GIFT_Change_Percent'
}, inplace=True)
df_Gnifty['Date'] = pd.to_datetime(df_Gnifty['Date'], format='%d %b %Y')
df_Gnifty.head()

,Date,GIFT_Open,GIFT_Close,Change(%),GIFT_High,GIFT_Low
0,2025-10-13,"25,310.50","25,305.00",-0.02%,"25,355.00","25,226.00"
1,2025-10-12,"25,205.00","25,205.00",-,"25,205.00","25,205.00"
2,2025-10-11,"25,205.00","25,205.00",-,"25,205.00","25,205.00"
3,2025-10-10,"25,227.00","25,205.00",-0.09%,"25,468.00","25,205.00"
4,2025-10-09,"25,131.50","25,222.50",0.36%,"25,309.00","25,120.50"


In [7]:
df = pd.merge(df_Nifty, df_Gnifty, on='Date', how='inner')
display(df)

,Date,Close,High,Low,Open,Volume,RSI,MACD,MACD_Signal,EMA_50,SMA_20,StochRSI_K,StochRSI_D,GIFT_Open,GIFT_Close,Change(%),GIFT_High,GIFT_Low
0,2025-03-28,23519.349609,23649.199219,23450.199219,23600.400391,295400,NaN,0.000000,0.000000,23519.349609,NaN,NaN,NaN,"23,742.50","23,490.00",-1.06%,"23,782.50","23,490.00"
1,2025-04-01,23165.699219,23565.150391,23136.400391,23341.099609,314300,NaN,-28.211427,-5.642285,23505.480967,NaN,NaN,NaN,"23,510.00","23,293.00",-0.92%,"23,652.50","23,221.00"
2,2025-04-02,23332.349609,23350.000000,23158.449219,23192.599609,341000,NaN,-36.698851,-11.853599,23498.691502,NaN,NaN,NaN,"23,297.50","23,133.00",-0.71%,"23,553.00","23,125.00"
3,2025-04-03,23250.099609,23306.500000,23145.800781,23150.300781,283200,NaN,-49.491575,-19.381194,23488.942800,NaN,NaN,NaN,"23,089.50","23,226.50",0.59%,"23,392.00","23,011.50"
4,2025-04-04,22904.449219,23214.699219,22857.449219,23190.400391,466800,NaN,-86.523609,-32.809677,23466.021483,NaN,NaN,NaN,"23,199.50","22,343.50",-3.69%,"23,240.50","22,313.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,2025-10-07,25108.300781,25220.900391,25076.300781,25085.300781,288000,45.623119,11.537397,18.230415,24910.331372,25024.359961,0.171565,0.139986,"25,152.00","25,217.00",0.26%,"25,309.00","25,145.50"
128,2025-10-08,25046.150391,25192.500000,25008.500000,25079.750000,227400,40.305099,18.253080,18.234948,24915.657608,25033.237500,0.157538,0.164701,"25,224.50","25,123.00",-0.40%,"25,284.00","25,077.50"
129,2025-10-09,25181.800781,25199.250000,25024.300781,25074.300781,286300,41.980149,34.127757,21.413510,24926.094595,25043.672559,0.114772,0.147958,"25,131.50","25,222.50",0.36%,"25,309.00","25,120.50"
130,2025-10-10,25285.349609,25330.750000,25156.849609,25167.650391,232900,48.623271,54.436567,28.018121,24940.183027,25057.665039,0.160086,0.144132,"25,227.00","25,205.00",-0.09%,"25,468.00","25,205.00"


In [8]:

# Convert string columns to numeric by removing commas in df
for col in ['GIFT_Open', 'GIFT_Close']:
    df[col] = df[col].astype(str).str.replace(',', '', regex=False)
    df[col] = pd.to_numeric(df[col])

feature_cols = ['Open','Close', 'RSI', 'MACD', 'MACD_Signal', 'EMA_50', 'SMA_20', 'StochRSI_K', 'StochRSI_D', 'GIFT_Open', 'GIFT_Close']

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[feature_cols])

# Prepare the input for prediction
x_input = np.array([scaled_data[-100:]])  
x_input = x_input.reshape(1, 100, 11)  

# Predict today's price
y_predicted_scaled = model.predict(x_input)

dummy_array = np.zeros(shape=(len(y_predicted_scaled), 11))

dummy_array[:, 0] = y_predicted_scaled[:, 0] # Predicted 'Close'
dummy_array[:, 1] = y_predicted_scaled[:, 1] # Predicted 'Open'

predicted_prices = scaler.inverse_transform(dummy_array)

# --- Display the results ---
print(f"Predicted Close Price for {now.date()}: {predicted_prices[0, 0]:.2f}")
print(f"Predicted Open Price for {now.date()}: {predicted_prices[0, 1]:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step
Predicted Close Price for 2025-10-14: 25230.09
Predicted Open Price for 2025-10-14: 25257.10
